## which noun extractor to use?
* I compared noun extractor in soynlp which has two version, v1 and v2.
* v2 compound version includes many ~한다
* v2 non-compound also include 한다
* In addition, v2 has much more noun.
* So I decided to use v1 with some filtering like "2글자 이상", "&영어", '#%' ...

In [27]:
import pymysql
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from konlpy.tag import Komoran, Kkma
import soynlp
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
import re

In [28]:
db1 = pymysql.connect(host='localhost', port=3306,user='*****', passwd='*****',
                        db='*****', charset='utf8' )

In [3]:
query1 = 'select news_content from kb_kor_news_code_clean1;'
df1 = pd.read_sql(query1, db1)
contents = df1['news_content'].tolist()

In [4]:
len(contents)

420358

whole news are too big to calculate.  
Chunk each 50000 news and get nouns

In [5]:
edge = range(0, len(contents), 50000)

contentss, steps = [], []
for i_x, i_r in enumerate(edge[1:]):
    i_l = edge[i_x]
    steps.append([i_l, i_r])
    
if steps[-1][1] < len(contents):
    steps[-1][1] = len(contents)
for i_l, i_r in steps:
    contentss.append( contents[i_l:i_r] )

In [6]:
corpuss = [ DoublespaceLineCorpus(content1, iter_sent=True) for content1 in contentss ]
#corpus = DoublespaceLineCorpus(contents, iter_sent=True)

In [7]:
# 가-힣 한글 범위
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=False)
whole_dic = {}
for corpus1 in corpuss:
    nouns = noun_extractor.train_extract(corpus1)
    for key1, val1 in nouns.items():
        #if len(key1)>1 and val1.frequency > 10 and val1.score > 0.4 and re.search("(&[a-zA-Z1-9]+|\d+\.*%|\d+.원|\d+억|\d+조)", key1) is None:
        if len(key1)>1 and val1.frequency > 10 and val1.score > 0.4 and re.search("[^가-힣]", key1) is None:
            whole_dic.setdefault(key1, 0)
            whole_dic[key1] += val1.frequency * val1.score

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 686738 from 933263 sents. mem=1.239 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=10172514, mem=2.962 Gb
[Noun Extractor] batch prediction was completed for 226032 words
[Noun Extractor] postprocessing detaching_features : 72746 -> 72440
[Noun Extractor] postprocessing ignore_features : 72440 -> 72364
[Noun Extractor] postprocessing ignore_NJ : 72364 -> 72028
[Noun Extractor] 72028 nouns (0 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=3.307 Gb                    
[Noun Extractor] 79.31 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 663963 from 896662 sents. mem=3.480 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=9794859

In [19]:
import pickle as pk
f1_s = "corpus/sejong_corpus_cleaner/scripts/word.pk"
with open(f1_s, 'rb') as f1:
    dic1 = pk.load(f1)

In [20]:
dic_keys = []
for noun_tag in dic1.keys():
    elms = noun_tag.split('\t')
    if len(elms) > 1:
        dic_keys.append(elms[0])

In [21]:
unknown_noun = {}
ii = 0
for key1, val1 in whole_dic.items():
    if key1 not in dic_keys:
        unknown_noun[key1] = val1
        if ii % 10000 == 0:
            print(ii, key1)
    if ii % 5000 == 0:
        print('---', ii)
    ii += 1

0 메타헬스케어투자조합
--- 0
--- 5000
--- 10000
--- 15000
20000 가족여행
--- 20000
--- 25000
30000 영화산업
--- 30000
--- 35000
40000 이어업
--- 40000


In [22]:
print(len(whole_dic))
import operator
sorted_dic = sorted(unknown_noun.items(), key=operator.itemgetter(1), reverse=True)
print(len(sorted_dic))

44966
28218


compare with dictionary

write obtained new nouns into user dictionary

In [23]:
f1_s = '/home/jupyter0/work/corpus/sejong_corpus_cleaner/scripts/user.dic'
with open(f1_s, 'w') as f1:
    for x_val in sorted_dic:
        noun1 = x_val[0]
        f1.write(noun1 + '\n')

write obtained new nouns into Komoran dictionary

In [26]:
f1_s = '/home/jupyter0/work/corpus/sejong_corpus_cleaner/scripts/dic2.word'
with open(f1_s, 'a') as f1:
    for x_val in sorted_dic:
        noun1 = x_val[0] + '\tNNG:' + str(int(x_val[1])) + '\n'
        #print(noun1)
        f1.write(noun1)

In [27]:
arr1 = sorted(list(filter(lambda x: len(x) > 1, nounsb.keys())))